In [157]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import random

**Download Data**

In [158]:
data = yf.download("NVDA", start="2018-01-01", end="2023-01-01")

data["Return"] = data["Close"].pct_change()
data.dropna(inplace=True)

data["Target"] = (data["Return"] > 0).astype(int)
print(data)

C:\Users\Kyllian\AppData\Local\Temp\ipykernel_28248\4273540099.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("NVDA", start="2018-01-01", end="2023-01-01")
[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open     Volume    Return  \
Ticker           NVDA       NVDA       NVDA       NVDA       NVDA             
Date                                                                          
2018-01-03   5.253192   5.283604   5.037596   5.046249  914704000  0.065814   
2018-01-04   5.280883   5.391154   5.258631   5.334535  583268000  0.005271   
2018-01-05   5.325635   5.362969   5.218825   5.295718  580124000  0.008474   
2018-01-08   5.488815   5.562988   5.404257   5.449256  881216000  0.030640   
2018-01-09   5.487332   5.533814   5.405741   5.494255  497000000 -0.000270   
...               ...        ...        ...        ...        ...       ...   
2022-12-23  15.191639  15.324513  14.868944  15.181648  349326000 -0.008671   
2022-12-27  14.107664  15.085740  14.042725  15.059764  464902000 -0.071353   
2022-12-28  14.022744  14.248531  13.870887  13.913847  351066000 -0.006019   
2022-12-29  14.589209  14.669133  14.213564  14.3883

**Make the model deterministic**

In [ ]:
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

**Build Features**

In [160]:
lookback = 5
X, y = [], []
for i in range(lookback, len(data)):
    X.append(data["Return"].values[i-lookback:i])
    y.append(data["Target"].values[i])

X = np.array(X)
y = np.array(y)

# Standardisation
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split train / test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

**MLP Model**

In [161]:
model = models.Sequential([
    layers.Dense(32, activation="relu", input_shape=(X_train.shape[1],)),
    # layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")  # sortie = proba
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

c:\Users\Kyllian\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


**Training**

In [162]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=20, batch_size=32, verbose=1)

Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5299 - loss: 0.6976 - val_accuracy: 0.4781 - val_loss: 0.7470
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5389 - loss: 0.6916 - val_accuracy: 0.4741 - val_loss: 0.7437
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5389 - loss: 0.6886 - val_accuracy: 0.4741 - val_loss: 0.7432
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5469 - loss: 0.6867 - val_accuracy: 0.4462 - val_loss: 0.7436
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5479 - loss: 0.6853 - val_accuracy: 0.4542 - val_loss: 0.7443
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5489 - loss: 0.6843 - val_accuracy: 0.4382 - val_loss: 0.7448
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5469 - loss: 0.6835 - val_accuracy: 0.4422 - val_loss: 0.7452
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5489 - loss: 0.6827 - val_accuracy: 0.4382 - val_loss:

**Long Only Strategy**

In [163]:
proba = model.predict(X_test).flatten()
positions = (proba > 0.5).astype(int)  # 1 si on prend position

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


**Strategy  Simulation**

In [164]:
returns = data["Return"].iloc[-len(y_test):].values
strategy_returns = positions * returns

In [165]:
print("Rendement cumulé stratégie :", np.cumprod(1+strategy_returns)[-1] - 1)

Rendement cumulé stratégie : -0.40016312611415894
